In [26]:
from netCDF4 import Dataset
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, ALL_TIMES,interpline, CoordPair, xy_to_ll, ll_to_xy)
import pandas as pd
import numpy as np
import os
import shutil
import re
import netCDF4 as nc
import xarray as xr

In [27]:
#mask out seas-1620

ds_lu_1620 = xr.open_dataset('wrfout_d04_2011-06-23_11:00:00')
lu_1620 = ds_lu_1620['LU_INDEX']

lu_1620 = lu_1620[0,:,:]

del lu_1620.coords['XTIME']

lu_1620_df = lu_1620.to_dataframe().reset_index()

In [28]:
#mask out seas-2040s

ds_lu_2040s = xr.open_dataset('wrfout_d04_2040-06-18_23:00:00')
lu_2040s = ds_lu_2040s['LU_INDEX']

lu_2040s = lu_2040s[0,:,:]

del lu_2040s.coords['XTIME']

lu_2040s_df = lu_2040s.to_dataframe().reset_index()

# 2010s

In [29]:
ds1 = xr.open_dataset('ssp245_t2_1120_era5_corrected_dailymin.nc')
t2_1120_ssp245_dailymin = ds1['ssp245_t2_1120_era5_corrected_dailymin']

t2_1120_ssp245_dailymin_df = t2_1120_ssp245_dailymin.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_1120_dailymin_thres = (t2_1120_ssp245_dailymin_df[t2_1120_ssp245_dailymin_df['ssp245_t2_1120_era5_corrected_dailymin']>=28]).reset_index(drop=True)

In [30]:
t2_1120_dailymin_thres = t2_1120_dailymin_thres.sort_values(['XLAT','XLONG','Time'], ascending=True)

t2_1120_dailymin_thres_latlon = t2_1120_dailymin_thres.groupby(['XLAT','XLONG'])


t2_1120_dailymin_thres['auxi_col'] = t2_1120_dailymin_thres_latlon['Time'].rank()


t2_1120_dailymin_thres['date_sub'] = t2_1120_dailymin_thres['Time'] - pd.to_timedelta(t2_1120_dailymin_thres['auxi_col'],unit='d')

HND_1120 = t2_1120_dailymin_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub']).count().reset_index()
HND_1120 = HND_1120.rename(columns={'auxi_col':'concecutive_hot_nights_num'})

HND_1120 = HND_1120[HND_1120['concecutive_hot_nights_num']>=3]

HND_1120 = HND_1120.groupby(['XLAT','XLONG','south_north','west_east'])['concecutive_hot_nights_num'].mean().reset_index()

In [31]:
#mask out the sea

HND_1120 = pd.merge(HND_1120,lu_1620_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HND_1120['concecutive_hot_nights_num'] = HND_1120['concecutive_hot_nights_num'].fillna(0)

HND_1120.loc[HND_1120.LU_INDEX == 16, 'concecutive_hot_nights_num'] = 9999

HND_1120 = HND_1120.rename(columns={"concecutive_hot_nights_num": "HND"})

In [32]:
HND_1120.to_csv('HND_1120.csv',index=False,columns=['XLAT','XLONG','HND'])

HND_1120.to_csv('HND_1120_withgrids.csv',index=False,columns=['XLAT','XLONG','HND','south_north','west_east'])

# 2040s

In [33]:
ds2 = xr.open_dataset('ssp245_t2_4049_era5_corrected_dailymin.nc')
t2_4049_ssp245_dailymin = ds2['ssp245_t2_4049_era5_corrected_dailymin']

t2_4049_ssp245_dailymin_df = t2_4049_ssp245_dailymin.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_4049_dailymin_thres = (t2_4049_ssp245_dailymin_df[t2_4049_ssp245_dailymin_df['ssp245_t2_4049_era5_corrected_dailymin']>=28]).reset_index(drop=True)

In [34]:
t2_4049_dailymin_thres = t2_4049_dailymin_thres.sort_values(['XLAT','XLONG','Time'], ascending=True)


t2_4049_dailymin_thres_latlon = t2_4049_dailymin_thres.groupby(['XLAT','XLONG'])


t2_4049_dailymin_thres['auxi_col'] = t2_4049_dailymin_thres_latlon['Time'].rank()


t2_4049_dailymin_thres['date_sub'] = t2_4049_dailymin_thres['Time'] - pd.to_timedelta(t2_4049_dailymin_thres['auxi_col'],unit='d')


HND_4049 = t2_4049_dailymin_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub']).count().reset_index()
HND_4049 = HND_4049.rename(columns={'auxi_col':'concecutive_hot_nights_num'})


HND_4049 = HND_4049[HND_4049['concecutive_hot_nights_num']>=3]


HND_4049 = HND_4049.groupby(['XLAT','XLONG','south_north','west_east'])['concecutive_hot_nights_num'].mean().reset_index()

In [35]:
#mask out the sea

HND_4049 = pd.merge(HND_4049,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HND_4049['concecutive_hot_nights_num'] = HND_4049['concecutive_hot_nights_num'].fillna(0)

HND_4049.loc[HND_4049.LU_INDEX == 16, 'concecutive_hot_nights_num'] = 9999

HND_4049 = HND_4049.rename(columns={"concecutive_hot_nights_num": "HND"})

In [36]:
HND_4049.to_csv('HND_4049.csv',index=False,columns=['XLAT','XLONG','HND'])

HND_4049.to_csv('HND_4049_withgrids.csv',index=False,columns=['XLAT','XLONG','HND','south_north','west_east'])

# 2090s

In [37]:
ds2 = xr.open_dataset('ssp245_t2_9099_era5_corrected_dailymin.nc')
t2_9099_ssp245_dailymin = ds2['ssp245_t2_9099_era5_corrected_dailymin']

t2_9099_ssp245_dailymin_df = t2_9099_ssp245_dailymin.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_9099_dailymin_thres = (t2_9099_ssp245_dailymin_df[t2_9099_ssp245_dailymin_df['ssp245_t2_9099_era5_corrected_dailymin']>=28]).reset_index(drop=True)

In [38]:
t2_9099_dailymin_thres = t2_9099_dailymin_thres.sort_values(['XLAT','XLONG','Time'], ascending=True)


t2_9099_dailymin_thres_latlon = t2_9099_dailymin_thres.groupby(['XLAT','XLONG'])


t2_9099_dailymin_thres['auxi_col'] = t2_9099_dailymin_thres_latlon['Time'].rank()


t2_9099_dailymin_thres['date_sub'] = t2_9099_dailymin_thres['Time'] - pd.to_timedelta(t2_9099_dailymin_thres['auxi_col'],unit='d')


HND_9099 = t2_9099_dailymin_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub']).count().reset_index()
HND_9099 = HND_9099.rename(columns={'auxi_col':'concecutive_hot_nights_num'})


HND_9099 = HND_9099[HND_9099['concecutive_hot_nights_num']>=3]


HND_9099 = HND_9099.groupby(['XLAT','XLONG','south_north','west_east'])['concecutive_hot_nights_num'].mean().reset_index()

In [39]:
#mask out the sea

HND_9099 = pd.merge(HND_9099,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HND_9099['concecutive_hot_nights_num'] = HND_9099['concecutive_hot_nights_num'].fillna(0)

HND_9099.loc[HND_9099.LU_INDEX == 16, 'concecutive_hot_nights_num'] = 9999

HND_9099 = HND_9099.rename(columns={"concecutive_hot_nights_num": "HND"})

In [40]:
HND_9099.to_csv('HND_9099.csv',index=False,columns=['XLAT','XLONG','HND'])

HND_9099.to_csv('HND_9099_withgrids.csv',index=False,columns=['XLAT','XLONG','HND','south_north','west_east'])

# diff

In [41]:
HND_1120 = pd.read_csv('HND_1120_withgrids.csv')

HND_1120 = HND_1120.replace(9999,np.nan)

HND_1120 = HND_1120.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [42]:
HND_4049 = pd.read_csv('HND_4049_withgrids.csv')

HND_4049 = HND_4049.replace(9999,np.nan)

HND_4049 = HND_4049.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [43]:
HND_9099 = pd.read_csv('HND_9099_withgrids.csv')

HND_9099 = HND_9099.replace(9999,np.nan)

HND_9099 = HND_9099.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [44]:
#test

print((HND_1120['west_east']-HND_4049['west_east']).min())
print((HND_1120['west_east']-HND_4049['west_east']).max())
print((HND_1120['south_north']-HND_4049['south_north']).max())
print((HND_1120['south_north']-HND_4049['south_north']).min())
print((HND_4049['west_east']-HND_9099['west_east']).max())
print((HND_4049['west_east']-HND_9099['west_east']).min())
print((HND_4049['south_north']-HND_9099['south_north']).max())
print((HND_4049['south_north']-HND_9099['south_north']).min())

0
0
0
0
0
0
0
0


In [45]:
HND_diff_2040s = HND_1120.copy()
HND_diff_2090s = HND_1120.copy()

In [46]:
HND_diff_2040s['diff'] = HND_4049['HND']-HND_1120['HND']

HND_diff_2090s['diff'] = HND_9099['HND']-HND_1120['HND']

In [47]:
HND_diff_2040s['diff'] = HND_diff_2040s['diff'].fillna(9999)

HND_diff_2090s['diff'] = HND_diff_2090s['diff'].fillna(9999)

In [48]:
HND_diff_2040s.to_csv('HND_diff_2040s.csv')
HND_diff_2090s.to_csv('HND_diff_2090s.csv')